In [3]:
from regain.datasets.hmm import generate_hmm
from regain.hmm.hmm_graphical_lasso import HMM_GraphicalLasso 
import numpy as np
from importlib import reload
import regain.hmm.utils; reload(regain.hmm.utils)
from regain.hmm.utils import results_recap
from regain.hmm.utils import cross_validation#, cross_validation_auto
import pickle as pkl
from sklearn import cluster
from sklearn.mixture import GaussianMixture



/Users/vt908/opt/anaconda3/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/vt908/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/vt908/opt/anaconda3/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/vt908/opt/anaconda3/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: nump

In [4]:
# list of parameters to generate different kind of datasets

states = [2,5,10,15]
dims = [10, 15, 20, 30]
mode_prec = ['regain','regain_random']
mode_mea = ['Normal']#, 'Uniform']

res_all = {}

In [5]:
from sklearn.covariance import empirical_covariance
from regain.covariance import GraphicalLasso

def compute_precision(X,K,clus, alpha):
    
    precisions = []
    N, d = X.shape
    means = np.zeros((K, d))
    
    for i, l in enumerate(np.unique(clus)):
        
        means[i, :] = np.mean(X[np.where(clus == l)[0], :], axis=0)
        emp_cov = empirical_covariance(X - means[i, :],assume_centered=True)
        gl = GraphicalLasso(alpha=alpha)
        precisions.append(gl.fit(emp_cov).precision_)
    
    return precisions
    

In [ ]:
for st in states:
    for dim in dims:
        for mm in mode_prec:


            print('Case: N_states:',st,',TS dim:',dim,',Prec mode:',mm)

            # Generate a dataset
            dataset = generate_hmm(n_samples=1000,
                                   n_states=st,
                                   n_dim_obs=dim,
                                   mode_precisions=mm,
                                   mode_mean = 'Normal',
                                   transition_type='fixed',
                                   sigma=1)
            # CV hmm_gmm 

            hmm_gmm = HMM_GraphicalLasso(alpha=1,
                                         max_iter = 1000,
                                         n_clusters=st,
                                         verbose=False,
                                         mode='scaled',
                                         warm_restart=True,
                                         repetitions=5,
                                         n_jobs=-1)


            if st == 2:
                alpha_list = np.linspace(35,115 ,20)
            elif st == 5:
                alpha_list = np.linspace(25,55 ,20)
            elif st == 10:
                alpha_list = np.linspace(15,45 ,20)
            elif st == 15:
                alpha_list = np.linspace(5,35 ,20)


            res = cross_validation(hmm_gmm,
                   dataset['data'],
                   params={'alpha':alpha_list,
                           'n_clusters':[st]},
                   n_repetitions=1)

            # define several different models

            hmm_gmm = HMM_GraphicalLasso(alpha=res[0][0], n_clusters=st, verbose=False, mode='scaled',
                                     warm_restart=True, repetitions=50, n_jobs=-1)
            
            #hmm
            
            hmm = HMM_GraphicalLasso(alpha=0, n_clusters=st, verbose=False, mode='scaled',
                                     warm_restart=True, repetitions=5, n_jobs=-1)

            
            # gmm 
            gmm = GaussianMixture(n_components=st)

            
            spectral = cluster.SpectralClustering(n_clusters=st,assign_labels="discretize")
            
            # Kmeans
            kmeans = cluster.KMeans(n_clusters=st)
            

            # fit and save results
            try :
                hmm_gmm.fit(dataset['data'])
                res_all[(st, dim, mm, 'HMM_GMM')] = results_recap(dataset['states'], 
                                                                   hmm_gmm.labels_,
                                                                   dataset['thetas'], 
                                                                   hmm_gmm.precisions_)
            except:
                print("HMM-GMM failed")

            
            try :
                hmm.fit(dataset['data'])
                res_all[(st, dim, mm, 'HMM')] = results_recap(dataset['states'], 
                                                                   hmm.labels_,
                                                                   dataset['thetas'], 
                                                                   hmm.precisions_)
            except:
                print("HMM failed")
            
            try :
                gmm.fit(dataset['data'])
                res_all[(st, dim, mm, 'GMM')] = results_recap(dataset['states'], 
                                                              gmm.predict(dataset['data']),
                                                              dataset['thetas'], 
                                                              gmm.precisions_)
            except:
                print("GMM failed")

            
            try :
                spectral.fit(dataset['data'])
                res_all[(st, dim, mm, 'spectral')] = results_recap(dataset['states'], 
                                                                   spectral.labels_,
                                                                   dataset['thetas'], 
                                                                   compute_precision(dataset['data'],
                                                                                     st,
                                                                                     spectral.labels_, res[0][0]))
            except:
                print("Spectral failed")


            
            try :
                kmeans.fit(dataset['data'])
                res_all[(st, dim, mm, 'kmeans')] = results_recap(dataset['states'], 
                                                                  kmeans.labels_,
                                                                  dataset['thetas'], 
                                                                  compute_precision(dataset['data'],
                                                                                    st,
                                                                                    kmeans.labels_, res[0][0]))
            except:
                print("Kmeans failed")

                



  0%|          | 0/1 [00:00<?, ?it/s]

Case: N_states: 2 ,TS dim: 10 ,Prec mode: regain



100%|██████████| 20/20 [03:39<00:00, 10.98s/it]
/Users/vt908/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:834: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/vt908/Documents/git_repos/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective did not converge.
  warnings.warn("Objective did not converge.")
/Users/vt908/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:834: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/vt908/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:834: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
  0%|          | 0/1 [00:00<?, ?it/s]

Case: N_states: 2 ,TS dim: 10 ,Prec mode: regain_random



100%|██████████| 20/20 [01:04<00:00,  3.21s/it]
/Users/vt908/Documents/git_repos/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective did not converge.
  warnings.warn("Objective did not converge.")
/Users/vt908/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:834: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/vt908/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:834: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


...

100%|██████████| 20/20 [06:13<00:00, 18.69s/it]
/Users/vt908/Documents/git_repos/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective did not converge.
  warnings.warn("Objective did not converge.")
/Users/vt908/Documents/git_repos/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective did not converge.
  warnings.warn("Objective did not converge.")
/Users/vt908/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:834: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/vt908/Documents/git_repos/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective did not converge.
  warnings.warn("Objective did not converge.")
/Users/vt908/Documents/git_repos/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective did not converge.
  warnings.warn("Objective did not converge.")
/Users/vt908/opt/anaconda3/envs/py36/lib/python3.6/site

Case: N_states: 5 ,TS dim: 15 ,Prec mode: regain_random


100%|██████████| 20/20 [03:55<00:00, 11.76s/it]
/Users/vt908/Documents/git_repos/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective did not converge.
  warnings.warn("Objective did not converge.")
/Users/vt908/Documents/git_repos/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective did not converge.
  warnings.warn("Objective did not converge.")
/Users/vt908/Documents/git_repos/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective did not converge.
  warnings.warn("Objective did not converge.")
/Users/vt908/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:834: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/vt908/Documents/git_repos/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective did not converge.
  warnings.warn("Objective did not converge.")
/Users/vt908/Documents/git_repos/regain/regain/covarian

Case: N_states: 5 ,TS dim: 20 ,Prec mode: regain


100%|██████████| 20/20 [1:04:55<00:00, 194.80s/it]
